# Simulation of the Process with Input Variations

Import PyRolL modules.

In [1]:
import pyroll.basic as pr
import pyroll.export as pre

Import the process data.

In [2]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE, TEMPERATURE, DIAMETER, TEMPERATURE_STD, DIAMETER_STD, create_in_profile
from weiner_variation.sim.config import SAMPLE_COUNT, SEED
from weiner_variation.sim.data import DrawInput
from weiner_variation.config import DATA_DIR

Import further libraries.

In [3]:
import pandas as pd
from copy import deepcopy
from scipy.stats import norm
import numpy as np
import tqdm
from multiprocessing import Pool

Parameters for notebook using papermill.

In [4]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = DATA_DIR / "sim_input_results.csv"

In [5]:
# Parameters
OUTPUT_FILENAME = "/home/maxwe/Git/pyroll/publication-weiner-variation/weiner_variation/data/sim_diameter_stds_results/0.1.csv"
DIAMETER_STD = 0.005000000000000001


Create distribution functions for input values.

In [6]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

Draw random inputs from distributions.

In [7]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

Define a worker function creating the in profile, running the solution procedure and extracting results.

In [8]:
def worker(draw: DrawInput):    
    ip = create_in_profile(draw.diameter)
    ip.temperature = draw.temperature

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return pre.to_pandas(sequence)

Run the simulations using a process pool.

In [9]:
results = list(tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT))

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:05,  1.55it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.52it/s]

100%|██████████| 10/10 [00:01<00:00,  7.84it/s]

Create a dataframe from results.

In [10]:
df = pd.DataFrame([r.stack().swaplevel().sort_index() for r in results])
df

abs_draught                                                              \
           0         2         4         6         8         10        12   
0   -0.016498 -0.021420 -0.014494 -0.019903 -0.009272 -0.011220 -0.007985   
1   -0.017912 -0.022786 -0.014563 -0.019938 -0.009279 -0.011223 -0.007986   
2   -0.020101 -0.024797 -0.014646 -0.019980 -0.009287 -0.011227 -0.007986   
3   -0.015038 -0.019939 -0.014405 -0.019858 -0.009264 -0.011216 -0.007984   
4   -0.017971 -0.022840 -0.014565 -0.019939 -0.009279 -0.011223 -0.007986   
5   -0.019711 -0.024439 -0.014633 -0.019973 -0.009285 -0.011226 -0.007986   
6   -0.015797 -0.020720 -0.014454 -0.019883 -0.009268 -0.011218 -0.007985   
7   -0.015618 -0.020538 -0.014443 -0.019878 -0.009267 -0.011218 -0.007985   
8   -0.008451 -0.010577 -0.013005 -0.019128 -0.009120 -0.011149 -0.007972   
9   -0.014319 -0.019132 -0.014348 -0.019830 -0.009258 -0.011214 -0.007984   

                                 ... zouhar_contact_in_width            \
         14        16        18  ...                      8         10   
0 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005700   
1 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005699   
2 -0.010436 -0.004337 -0.005911  ...                   0.026  0.005697   
3 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
4 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005699   
5 -0.010436 -0.004337 -0.005911  ...                   0.026  0.005697   
6 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
7 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
8 -0.010429 -0.004335 -0.005910  ...                   0.026  0.005734   
9 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005703   

                                                                        
       12        14      16        18     20        22    24        26  
0  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
1  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
2  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
3  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
4  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
5  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
6  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
7  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
8  0.0198  0.003257  0.0148  0.004103  0.012  0.003359  0.01  0.002245  
9  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  

[10 rows x 3891 columns]

Save dataframe to CSV file.

In [11]:
df.to_csv(OUTPUT_FILENAME)